# Building an ETL Pipeline for WRC Hackathon

### Tools

In [29]:
# Importing Packages

import tweepy as tw         
import pandas as pd     
import numpy as np
from datetime import date, timedelta
import json
import re
from IPython.display import display
import pyodbc

In [30]:
# API Keys

CONSUMER_KEY    = 'egoItCf7bZMp4gopTCXC0WH2O'
CONSUMER_SECRET = 'Ph7fyNWzMnLTb0rgx5w0GPb0bzWlfyUtfVhZZGt5jVk4FJeBxK'

ACCESS_TOKEN  = '840149748354965504-6fGpkvdj6n53uVG5231Oq6PhyLzHlfO'
ACCESS_SECRET = 'f17t2HIfmpsgh1IBxgdugigEH8Xuzhps7gjGT2jfLOgxT'

In [31]:
# Create authentication

auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

### Data

Both sets need expansion

In [32]:
 key_words = [ 'leaks' , 'burst' , 'water pressure', 'water cuts', 'pipe' ,
 'water pipe', 'fault', 'broken', 'damaged', 'toilets', 'sanitation',
 'leakage', 'wasted', 'complaints', 'complained', 'reported', 'water supply',
 'service delivery', 'water outage', 'no water', 'reference number', 'water faults'
 'faults' ,'leaks' , 'report' , 'complain' ,'install','no']

In [33]:
master_list =[['alfred_enzo' ,'@alfrednzoDM' ,'amatola_water', 'eastern_cape', 'nelson_mandela_bay'],
              ['amajuba' ,'@AmajubaDM' , 'mhlathuze Water' , 'kwazulu_natal','ethekwini' ],
              ['cape_wine_lands','@CWDM2','overberg Water board','western_cape' , 'city_of_cape_town'],
              ['ehlanzeni' ,'@EhlanzeniM','dwa','mpumalanga' , 'ehlanzeni'],
              ['fezile_dabi', '@FezileDabiDM','dwa','free_state' , 'mangaung'],
              ['garden_route', '@GardenRoute_DM','dwa','western_cape' ,'city_of_cape_town'],
              ['gert_sibande','@GertSibandeDM','dwa','mpumalanga' , 'ehlanzeni'],
              ['ilembe' ,'@DistrictIlembe','umngeni_water','kwazulu_natal' , 'ethekwini'],
              ['king_cetshwayo','@Cetshwayo_dm','mhlathuze_water','kwazulu_natal' , 'ethekwini'],
              ['lejweleputswa', '@LejweleputswaM','dws','free_state' , 'mangaung'],
              ['mangaung', '@MangaungCity','bloem_water','free_state' , 'mangaung'],
              ['nkangala', '@NkangalaDM','dws','mpumalanga' , 'mangaung'],
              ['sedibeng', '@SedibengDM','sedibeng_water','gauteng' , 'city_of_johannesburg'],
              ['ugu', '@UguDM','dws','kwazulu_natal' , 'ethekwini'],
              ['north_uthungulu','@UmkhanyakudeM','mhlathuze_water','kwazulu_natal' ,'ethekwini'],
              ['uthukela', '@OkhahlambaLM','umngeni_water','kwazulu_natal' ,'ethekwini'],
              ['vhembe', '@VhembeDM','dws','limpopo' , 'vhembe'],
              ['rand_west_city', '@RandLocal','dws','gauteng' , 'city_of_johannesburg'],
              ['sedibeng' ,'@EmfuleniLM','sedibeng_water','gauteng' , 'city_of_johannesburg'],
              ['amathole','Amathole District'] , 
              [ 'bojanala','Bojanala District'], 
              [ 'capricon' ,'Capricorn District'], 
              ['karoo' ,'Central Karoo District' ],
               ['chris_hani' ,'Chris Hani District'], 
              ['kenneth_kaunda','Dr Kenneth Kaunda District'], 
              ['dr_ruth','Dr Ruth Segomotsi Mompati District'] ,
              ['franses','Frances Baard District' ],
              ['harry_gwala','Harry Gwala District'], 
              ['joe_gqabi','Joe Gqabi District' ],
              ['john_taolo','John Taolo Gaetsewe District'], 
              ['mopani','Mopani District' ],
              ['namakwa','Namakwa District'], 
              ['ngaka','Ngaka Modiri Molema'], 
              ['or_tambo' ,'OR Tambo District'], 
              ['overberge','Overberg District' ],
              ['pixley_ka_seme','Pixley ka Seme District'], 
              ['sarah', 'Sarah Baartman District' ],
              ['sekhukune','Sekhukhune District' ],
              ['thabo','Thabo Mofutsanyana District'] ,
              ['umgungundlove' ,'uMgungundlovu District'] ,
              ['umzinyathi','uMzinyathi District' ],
              ['waterberg','Waterberg District' ],
              ['west_cost' ,'West Coast District'], 
              ['zf_mchunu' ,'ZF Mgcawu District' ],
              ['xhariep' ,'Xhariep District' ],
              ['zululand' ,'Zululand District']]

### Functions

In [34]:
def extract_tweets(search_word) :

    search_words = search_word + " -filter:retweets"
    today = date.today()                     
    tweets = tw.Cursor(api.search,
               q=search_words,
               lang="en"
               ).items(200)                    
    tweet_status = [tweet for tweet in tweets]
    tweet_list = [tweet.text for tweet in tweet_status]
    tweet_date = [tweet.created_at for tweet in tweet_status]
    co_od = [tweet.coordinates for tweet in tweet_status]

    tweet_df = pd.DataFrame(data=tweet_list, 
        columns=['message'])
    tweet_df['timestamp'] = tweet_date
        
    return tweet_df

In [35]:
def TweetCleaner(tweet):

    tweet = tweet.lower()  
    tweet = re.sub('@[\w]*','',tweet)  
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r'#\w*', '', tweet)     
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ') 
    
    return tweet

In [36]:
def create_df(the_list) :
    
    data = [['a', 'b' ,'c']]    
    dummy_df = pd.DataFrame(data, columns = ['message', 'timestamp','district'])

    for i in the_list :
        handle = i[1]
        df = extract_tweets(handle)
        df['message'] = df['message'].apply(TweetCleaner)
        #df['province'] = i[3]
        #df['metro'] = i[4]
        df['district'] = i[0]
        #df['water_board'] = i[2]

        dummy_df = dummy_df.append(df)
    
    dummy_df = dummy_df.drop([0])
    dummy_df = dummy_df.reset_index()
    
    #res = [keys for keys in key_words if(keys in dummy_df['message'])] # Suppose to filter key words
        
    
    return dummy_df


### Build Data

In [37]:
database = create_df(master_list)

In [38]:
database

,index,message,timestamp,district
0,1,on covid 19 talk mr ndyebo kopo from gcis_ecap...,2020-07-08 08:30:48,alfred_enzo
1,1,i have no doubt this was human e…,2020-07-09 16:54:59,amajuba
2,2,lmaoo ayibo is it in line with your communicat...,2020-07-09 16:51:01,amajuba
3,3,offices closed today until further notice due ...,2020-07-07 20:31:00,amajuba
4,4,good morning please install water meters for w...,2020-07-06 05:21:52,amajuba
...,...,...,...,...
478,3,123 xhariep district mayor mongi ntwanambi’s f...,2020-07-03 07:47:01,xhariep
479,4,must improve as xhariep district municipality ...,2020-07-02 21:17:24,xhariep
480,1,zululand district municipality offices tempora...,2020-07-03 10:52:03,zululand
481,2,zululand district municipality offices tempora...,2020-07-03 07:36:28,zululand


### Database connection

In [39]:
#### Not intergrated yet ####
.

SyntaxError: invalid syntax (<ipython-input-39-054d67dececc>, line 2)

In [ ]:
###---Setting up connection---###
###---Extracting old table from SQL---###
from sqlalchemy import create_engine
import urllib
conn = pyodbc.connect(driver='{SQL Server}',
                          host='ENTER_HOST',                        ### Displayed when you log into SQL  
                          database='Eskom_Database',                ### Make sure name matches the one used in SQL
                          trusted_connection='tcon',
                          user='sa')
twitter_table = pd.read_sql_query('select * from dbo.twitter_table',conn)

In [ ]:
def pyodbc_twitter(connection, df, twitter_table):
    """
    Function takes the dataframe returned by extract_municipality_hashtags() , Update
    the old twitter table in SQL using the pyodbc connection
    
    Args :
        Connection : A connection between python and SQL Server using pyodbc module
        df : Pandas Dataframe , Latests tweets and dates along with municipalities and hashtags columns
        twitter_table : Pandas Dataframe , Old table available in the SQL database yet to be updated
    
    Returns :
         None
    
    """
    params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=ENTER_HOST;DATABASE=Eskom_Database;UID=sa;trusted_connection='tcon'")     ### Enter your host on 'Server' in string format                            
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    df = pd.merge(df_fresh_tweets,twitter_table, how='outer', left_on=['Tweets','Date','municipality','hashtags'], right_on = ['Tweets','Date','municipality','hashtags'])
    df.to_sql('twitter_table', con=engine , if_exists = 'replace' , index = False)    
    return None 
pyodbc_twitter(conn, df_fresh_tweets, twitter_table)